In [1]:
import pymongo
import spacy
import decouple
import glob
import datetime
from PIL import Image
from IPython.display import display
from tqdm.notebook import tqdm
from pytrends.request import TrendReq
import tweepy
import os
from io import BytesIO
from google.cloud import storage

config = decouple.AutoConfig(' ')
nlp = spacy.load("en_core_web_lg")

bucket_name = 'sideb-proejct.appspot.com'
bucket_dir = 'piixur/'

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.abspath(os.path.join(os.getcwd(), 'config/sideb-proejct-0e33d8c0b0a9.json'))

In [10]:
def get_gtrend(kw):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload([kw], timeframe='now 1-d')
    qrys = pytrends.related_queries()[kw]['rising']['query'].values
    return ' '.join([_q.replace(kw, '').replace('  ', ' ').strip() for _q in qrys])

In [11]:
get_gtrend('photo')

'robert crimo to pdf converter kolkata  result convert to pdf kolkata fort hanuman ji ki bajrangbali'

In [12]:
get_gtrend('art')

'metal of wisconsin provincia experta mother bird asoci cual es mi galeno prevencion museum fireworks 4th of july clip chelsea lee 4th of july pictures clip free rsl union 4th of july animal crossing guide museum fireworks 2022 animal crossing acnh guide coney walls reina sofia redd guide clip'

In [5]:
def get_mongodb_collection(colname):
    client = pymongo.MongoClient(config('MONGODB_URI'))
    return client.Cluster0[colname]

In [6]:
get_mongodb_collection('pix')

Collection(Database(MongoClient(host=['ac-w0gvyzx-shard-00-00.bhunbl6.mongodb.net:27017', 'ac-w0gvyzx-shard-00-01.bhunbl6.mongodb.net:27017', 'ac-w0gvyzx-shard-00-02.bhunbl6.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-13va1m-shard-0', tls=True), 'Cluster0'), 'pix')

In [7]:
def get_tweepy_api():
    bearer_token = config('TWITTER_BEARER_TOKEN')
    consumer_key = config('TWITTER_CONSUMER_KEY')
    consumer_secret = config('TWITTER_CONSUMER_SECRET')
    access_token = config('TWITTER_ACCESS_TOKEN')
    access_token_secret = config('TWITTER_ACCESS_TOKEN_SECRET')

    auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
    return tweepy.API(auth)

In [14]:
def main():
    log = {}
    
    try:
        gtrend = get_gtrend('photo')
        log['gtrend'] = gtrend

        colpix = get_mongodb_collection('pix')
        pixs = list(colpix.find({ 'feed_id': {'$exists': False}}))
        log['target_docs'] = len(pixs)

        sims = []
        for _p in tqdm(pixs):
            sims.append((_p['hash'], nlp(gtrend).similarity(nlp(_p['labels'] + ' ' + _p['colors']))))

        sims = sorted(sims, key=lambda x: -x[1])
        log['scores'] = sims[:3]

        _hash = sims[0][0]
        pix = colpix.find_one({ 'hash': _hash })
        url = pix['urls'][0]
        log['pix_url'] = url

        storage_client = storage.Client()
        blobs = list(storage_client.list_blobs(bucket_name, prefix=bucket_dir+_hash))
        blob = blobs[0]
        log['storage_file_exists'] = True

        api = get_tweepy_api()
        _file = BytesIO(blob.download_as_bytes())
        media = api.media_upload(filename='0', file=_file)
        log['twitter_media_id'] = media.media_id

        api.create_media_metadata(media.media_id_string, url)
        log['twitter_media_metadata_created'] = True

        status = api.update_status(status='', media_ids=[media.media_id_string])
        log['feed_id'] = status.id

        colpix.update_one(
            { 'hash': _hash }, 
            { '$set': { 'feed_id': status.id } }
        )
        log['mongodb_updated'] = True
        
    except:
        log['mongodb_updated'] = False
    
    finally:
        # print(log)
        feedlog = get_mongodb_collection('feedlog')
        feedlog.insert_one(log)

In [9]:
main()

  0%|          | 0/400 [00:00<?, ?it/s]